In [1]:
Correos_df = spark.sql("select cast(sender as string), bodyPreview, receivedDateTime, hasAttachments, importance, cast(subject as string), case when upper(bodyPreview) like '% UIS %' AND cast(sender as string) like '%uis%' OR cast(subject as string) like '%cursos%' OR upper(bodyPreview) like '%CORDIAL%' OR importance like '%high%' or hasAttachments then 1 else 0 end Fraude from correos700_json ")
Correos_df.createOrReplaceTempView("correos")
Correos_df.printSchema()
Correos_df.show(10)

root
-- sender: string (nullable = true)
-- bodyPreview: string (nullable = true)
-- receivedDateTime: string (nullable = true)
-- hasAttachments: boolean (nullable = true)
-- importance: string (nullable = true)
-- subject: string (nullable = true)
-- Fraude: integer (nullable = false)

+--------------------+--------------------+--------------------+--------------+----------+--------------------+------+
 sender| bodyPreview| receivedDateTime|hasAttachments|importance| subject|Fraude|
+--------------------+--------------------+--------------------+--------------+----------+--------------------+------+
[[divcult@uis.edu...|🎤🎭💃 - ¡Agenda ...|2019-05-13T12:43:16Z| false| normal|RV: 🎤🎭💃 - ¡Age...| 1|
[[uisdetodos@uis....|Reclama tu boleta...|2019-05-10T21:24:09Z| false| normal|HOY Santiago Bote...| 0|
[[no-reply@resear...|Juan Pablo, your ...|2019-05-10T14:42:11Z| false| normal|Juan Pablo, your ...| 0|
[[uisdetodos@uis....|#SemanaDeLaBicicl...|2019-05-10T11:59:51Z| false| normal|HOY Santiago Bote...| 0|
[[uisdetodos@uis....|#SemanaDeLaBicicl...|2019-05-09T22:37:31Z| false| normal|Invitación Taller...| 0|
[[uisdetodos@uis....|#SemanaDeLaBicicl...|2019-05-09T20:35:02Z| false| normal|HOY Ciclopaseo UI...| 0|
[[uisdetodos@uis....|#SemanaDeLaBicicl...|2019-05-09T18:31:30Z| false| normal|Santiago Botero e...| 0|
[[relext@uis.edu....|Oficina de Relaci...|2019-05-08T20:16:22Z| false| high|F.I. RELEXT - Mañ...| 1|
[[e3t@comunidadui...|Apreciada comunid...|2019-05-08T14:49:45Z| false| normal|E3TWeb Especializ...| 1|
[[uber.colombia@u...|Total: $12,200
W...|2019-05-08T13:45:40Z| false| normal|Your Wednesday mo...| 0|
+--------------------+--------------------+--------------------+--------------+----------+--------------------+------+
only showing top 10 rows

In [2]:
Correos_df = spark.sql("""select hasAttachments, importance, case when upper(bodyPreview) like '% UIS %' then 1 when upper(bodyPreview) like '% CORDIAL %' then 2 when upper(bodyPreview) like '% CORDIAL %' and upper(bodyPreview) like '% UIS %' then 3 else 0 end Body_UIS, case when sender like '%uis%' then 1 else 0 end Sender_UIS, case when subject like '%cursos%' then 1 when subject like '% uis %' then 2 else 0 end subject_U, Fraude from correos""")

In [3]:
(train, test) = Correos_df.randomSplit([0.8, 0.2], seed= 12345)
train.cache()
test.cache()

Out[ 3 ]: DataFrame[hasAttachments: boolean, importance: string, Body_UIS: int, Sender_UIS: int, subject_U: int, Fraude: int]

In [4]:
# Reset the DataFrames for no fraud (`dfn`) and fraud (`dfy`)
dfn = train.filter(train.Fraude == 0)
dfy = train.filter(train.Fraude == 1)

N = train.count()
y = dfy.count()
p = y/N

train_b = dfn.sample(False, p, seed = 92285).union(dfy)

display(train_b.groupBy("Fraude").count())

Fraude,count
1,107
0,79


In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

indexer = StringIndexer(inputCol = "importance", outputCol = "importanceIndexed")



va = VectorAssembler(inputCols = ["hasAttachments","importanceIndexed","Body_UIS","Sender_UIS","subject_U"], outputCol = "features")
dt = DecisionTreeClassifier(labelCol = "Fraude", featuresCol = "features", seed = 54321, maxDepth = 5,maxBins = 52)

In [6]:
pipeline_1 = Pipeline(stages=[ indexer, va, dt])

Model_b = pipeline_1.fit(train_b)
display(Model_b.stages[-1])

treeNode
"{""index"":1,""featureType"":""continuous"",""prediction"":null,""threshold"":0.5,""categories"":null,""feature"":3,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":0.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":9,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[0.0],""feature"":1,""overflow"":false}"
"{""index"":7,""featureType"":""continuous"",""prediction"":null,""threshold"":0.5,""categories"":null,""feature"":2,""overflow"":false}"
"{""index"":5,""featureType"":""continuous"",""prediction"":null,""threshold"":0.5,""categories"":null,""feature"":0,""overflow"":false}"
"{""index"":3,""featureType"":""continuous"",""prediction"":null,""threshold"":0.5,""categories"":null,""feature"":4,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":0.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":1.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":6,""featureType"":null,""prediction"":1.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":8,""featureType"":null,""prediction"":1.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"


In [7]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Use BinaryClassificationEvaluator to evaluate our model
evaluatorPR = BinaryClassificationEvaluator(labelCol = "Fraude", rawPredictionCol = "prediction", metricName = "areaUnderPR")
evaluatorAUC = BinaryClassificationEvaluator(labelCol = "Fraude", rawPredictionCol = "prediction", metricName = "areaUnderROC")

In [8]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Build the grid of different parameters
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.maxBins, [15, 20, 60]) \
    .build()

# Build out the cross validation
crossval = CrossValidator(estimator = dt,
                          estimatorParamMaps = paramGrid,
                          evaluator = evaluatorPR,
                          numFolds = 3)  

pipelineCV = Pipeline(stages=[indexer, va, crossval])

# Train the model using the pipeline, parameter grid, and preceding BinaryClassificationEvaluator
cvModel_u = pipelineCV.fit(train_b)

In [9]:
train_pred = cvModel_u.transform(train_b)
test_pred = cvModel_u.transform(test)
pr_train = evaluatorPR.evaluate(train_pred)
auc_train = evaluatorAUC.evaluate(train_pred)

In [10]:
# Create confusion matrix template
from pyspark.sql.functions import lit, expr, col, column

# Confusion matrix template
cmt = spark.createDataFrame([(1, 0), (0, 0), (1, 1), (0, 1)], ["Fraude", "prediction"])
cmt.createOrReplaceTempView("cmt")

In [11]:
# Source code for plotting confusion matrix is based on `plot_confusion_matrix` 
# via https://runawayhorse001.github.io/LearningApacheSpark/classification.html#decision-tree-classification
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, title):
  # Clear Plot
  plt.gcf().clear()

  # Configure figure
  fig = plt.figure(1)
  
  # Configure plot
  classes = ['Fraud', 'No Fraud']
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  # Normalize and establish threshold
  normalize=False
  fmt = 'd'
  thresh = cm.max() / 2.

  # Iterate through the confusion matrix cells
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  # Final plot configurations
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label') 
  
  # Display images
  image = fig
  
  # Show plot
  #fig = plt.show()
  
  # Save plot
  fig.savefig("confusion-matrix.png")

  # Display Plot
  display(image)
  
  # Close Plot
  plt.close(fig)

In [12]:
# Create temporary view for test predictions
test_pred.createOrReplaceTempView("test_pred")

Test_pred_cmdf = spark.sql("select a.Fraude, a.prediction, coalesce(b.count, 0) as count from cmt a left outer join (select Fraude, prediction, count(1) as count from test_pred group by Fraude, prediction) b on b.Fraude = a.Fraude and b.prediction = a.prediction order by a.Fraude desc, a.prediction desc")

# View confusion matrix
display(Test_pred_cmdf)

Fraude,prediction,count
1,1,30
1,0,2
0,1,0
0,0,118


In [13]:
cm_pdf = Test_pred_cmdf.toPandas()

# Create 1d numpy array of confusion matrix values
cm_1d = cm_pdf.iloc[:, 2]

# Create 2d numpy array of confusion matrix values
cm = np.reshape(cm_1d, (-1, 2))

# Print out the 2d array
print(cm)

/databricks/python/lib/python3.5/site-packages/numpy/core/fromnumeric.py:225: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
 return reshape(newshape, order=order)
[[ 30 2]
 [ 0 118]]

In [14]:
plot_confusion_matrix(cm, "Confusion Matrix ")

In [15]:
train_pred = Model_b.transform(train_b)
test_pred = Model_b.transform(test)
pr_train = evaluatorPR.evaluate(train_pred)
auc_train = evaluatorAUC.evaluate(train_pred)

# Evaluate the model on training datasets
pr_train = evaluatorPR.evaluate(train_pred)
auc_train = evaluatorAUC.evaluate(train_pred)

# Evaluate the model on test datasets
pr_test = evaluatorPR.evaluate(test_pred)
auc_test = evaluatorAUC.evaluate(test_pred)

# Print out the PR and AUC values
print("PR train:", pr_train)
print("AUC train:", auc_train)
print("PR test:", pr_test)
print("AUC test:", auc_test)

PR train: 0.9880916490804944
AUC train: 0.9719626168224299
PR test: 0.9754166666666667
AUC test: 0.96875

In [16]:
import mlflow
import mlflow.spark

import os
mlflow_experiment_id = 1030082808670079
with mlflow.start_run(experiment_id = mlflow_experiment_id) as run:
  # Log Parameters and metrics
  mlflow.log_param("balanced", "yes")
  mlflow.log_metric("PR train", pr_train)
  mlflow.log_metric("AUC train", auc_train)
  mlflow.log_metric("PR test", pr_test)
  mlflow.log_metric("AUC test", auc_test)
  
  # Log model
  mlflow.spark.log_model(Model_b, "model")
  
  # Log Confusion matrix
  mlflow.log_artifact("confusion-matrix.png")